In [ ]:
import numpy as np
import pandas as pd
import h5py
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, CustomJS, Slider, ColorBar, HoverTool, LinearColorMapper, TapTool
from bokeh.layouts import column, widgetbox, row
from bokeh.events import Tap, DoubleTap
import matplotlib.pyplot as plt
%matplotlib inline
output_notebook()
#output_file('Upconverting NP Superlattice Data Analysis.html',title='Upconverting NP Superlattice Data Analysis')

In [ ]:
import os, sys
#path = "C:\User\Sunli\Desktop\data\df-mic\20nm_d0"
FileNames = []
ps=dict()
map2D=dict()
for file in os.listdir():
    if file.endswith(".h5"):
       # print(os.path.join( file))
        FileNames.append(os.path.join(file))
for name in FileNames:

    if 'power' in name:
        power_scan=h5py.File(name,'r')
        ps['spectra']=np.array(power_scan['measurement/power_scan/spectra'])
        ps['wv']=np.array(power_scan['measurement/power_scan/wls'])
        ps['power']=np.array(power_scan['measurement/power_scan/pm_powers'])
    else:
        mapping=h5py.File(name,'r')
        map2D['spec_map']=np.array(mapping['/measurement/WinSpecMCL2DSlowScan/spec_map']).squeeze()
        map2D['spec_map_all']=np.sum(map2D['spec_map'],2)
        #map2D['spec_mean']=np.average(np.average(map2D['spec_map'],0),0)
        map2D['wv']=np.array(mapping['/measurement/WinSpecMCL2DSlowScan/wavelength'])
        map2D['dim_x']=np.array(mapping['/measurement/WinSpecMCL2DSlowScan/h_array'])
        map2D['dim_y']=np.array(mapping['/measurement/WinSpecMCL2DSlowScan/v_array'])
        map2D['dim_x']=map2D['dim_x']-np.min(map2D['dim_x'])
        map2D['dim_y']=map2D['dim_y']-np.min(map2D['dim_y'])

#**2D Map of Emission Intensity**
>First I'll explore the 2D distribution of emission from sample

In [ ]:
#################################################
wvl=750
source = ColumnDataSource({'mapping' : [map2D['spec_map'][:,:,wvl]]})
TOOLS = "save,pan,box_zoom,reset,wheel_zoom"  #hover,
tt='2D Map of Emission Intensity at Individual Wavelength'
left=figure(title=tt,tools=TOOLS,x_range=(0,20),y_range=(0,20),plot_width=400,plot_height=400,toolbar_location='below')
left.image(image='mapping',x=0,y=0,dw=20,dh=20,palette='Inferno256', source=source)
left.xaxis.axis_label = 'x dimension (um)'
left.yaxis.axis_label = 'y dimension (um)'
#to add color bar, check example: https://bokeh.pydata.org/en/latest/docs/gallery/unemployment.html
callback = CustomJS(args=dict({"source":source, "mapp":map2D['spec_map'],'wvl':map2D['wv'],
                               'xlength':len(map2D['dim_x']),'ylength':len(map2D['dim_y'])}),code="""
    var data=source.data
    var dat=mapp
    var swvl=wv_slider.value  
    var index = 0;
    var minVal = 1e10;
    wvl.forEach(function(val, i){
        var diff = Math.abs(val - swvl);
        if (diff < minVal){
            minVal = diff;
            index = i;
        }
    });   
    mapping = data['mapping'];
    //console.log("before", mapping, source.data)
    //console.log(swvl, index, wvl[index], xlength, ylength);
    var newData = []; 
    var i,j;
    for (j =0; j< ylength; j++) {
        for (i =0; i < xlength; i++) {
            newData.push(dat[i][j][index]); 
        }
   }
    source.data['mapping'] = [new Float64Array(newData)];
    source.change.emit();
""")
wv_slider = Slider(start=min(map2D['wv']),end=max(map2D['wv']),value=map2D['wv'][wvl],step=1,title='wavelength') #callback=callback
callback.args['wv_slider'] = wv_slider
wv_slider.js_on_change('value',callback)
layout=column(wv_slider,left)
#show(layout)

In [ ]:
####################################################
size=0
source_map = ColumnDataSource({'mapp' : [map2D['spec_map_all']]})
source_loc = ColumnDataSource(data=dict({'x1':[map2D['dim_x'][1]],'y1':[map2D['dim_y'][1]]}))  #,
ttl='2D Map of Emission Intensity (total)'
mid=figure(title=ttl,tools=TOOLS,x_range=(0,20),y_range=(0,20),plot_width=400,plot_height=400,toolbar_location='below')
hover = HoverTool()
hover.tooltips = [('(x,y)','($x,$y)'),('Note','Click to extract spectrum at this location')]
mid.image(image='mapp',x=0,y=0,dw=20,dh=20,palette='Inferno256',source=source_map)
mid.circle('x1','y1',source=source_loc,color='green',size=10,alpha=0.5)
mid.xaxis.axis_label = 'x dimension (um)'
mid.yaxis.axis_label = 'y dimension (um)'
mid.tools.append(hover)
#Need to learn how to save figures automatically
#from bokeh.io import export_png #export_png(mid,'2D Map of Emission Intensity.png')
#####################################################
source_total = ColumnDataSource(data=dict(x= map2D['wv'],y0=map2D['spec_map'][1,1,:])) #
ttle='Emission Spectrum at Selected Location'
right=figure(title=ttle,tools=TOOLS,plot_width=400,plot_height=400,toolbar_location='below') #y_range=(-10,max_y),max_y=np.max(map2D['spec_map'])
right.line('x','y0',source=source_total,color='green', line_width=2, line_alpha=0.4)
right.xaxis.axis_label = 'Wavelength (nm)'
right.yaxis.axis_label = 'Intensity (a.u.)'
#####################################################
callback1 = CustomJS(args=dict({"source_total":source_total,'source_loc':source_loc,'total' : map2D['spec_map'],'wvlength':len(map2D['wv']),'xd':map2D['dim_x'],'yd':map2D['dim_y']}),code="""
    var data=source_loc.data
    var tot=total 
    var xx=cb_obj.x
    var yy=cb_obj.y
    var x_idx,y_idx;
    //data['x1'].push(xx);
    //data['y1'].push(yy);
    data['x1']=[xx];
    data['y1']=[yy];
    //console.log(data['x1'],data['y1'])
    var minX = 100
    var minY = 100
    xd.forEach(function(val, i){
        var diff = Math.abs(val - xx);
        if (diff < minX){
            minX = diff;
            x_idx = i;
        }});
    yd.forEach(function(val, j){
        var diff = Math.abs(val - yy);
        if (diff < minY){
            minY = diff;
            y_idx = j;
        }}); 
    //console.log('xd',xd,'yd',yd,x_idx,y_idx);
    var newY=[]
    for (k=0;k<wvlength;k++){
    newY.push(tot[x_idx][y_idx][k])
    //console.log('tot',tot[x_idx][y_idx][k])
    }
    //console.log("total",source_total.data['y0'],"newY",newY)
    source_total.data['y0']=new Float64Array(newY)
    //console.log("new total",source_total.data['y0'])
    source_total.change.emit()
    source_loc.change.emit()
    
""")
mid.js_on_event(Tap,callback1)

In [ ]:
p=row(gridplot([wv_slider],[left],[mid,right]))
show(p)